In [ ]:
__author__ = 'kgeorge2@gmail.com'

# common notebook utils
### koshy george, kgeorge2@gmail.com

### In addition to [jupyter](http://jupyter.org/), you need to have [ipywidgets](https://github.com/ipython/ipywidgets) installed 

The contribution of this notebook are
1. define a custom ipywidget called ProgrEssImageWidget
2. define a handy Plotter class who can accepts samples in various channels and constuct a plot


### ProgressImageWidget

<code>ProgressImageWidget</code> will display any image assigned to its <code>value</code>, if the image is a [datauri](https://en.wikipedia.org/wiki/Data_URI_scheme) string. 

<code>
p=common.utils.ProgressImageWidget()
display(p)
</code>


..., lots of stuff


Now if if you assign the value element of <code>p</code> to some new image content, the widget displayed by the display  call above, will now have the new image content.
<code>
p.value = new_image_content_as_png_dataurl
</code>

Shown below is the sample output of <code>display(p)</code>
![progressimagewidget_demo](progressimagewidget_demo.png)

In [1]:

import ipywidgets as widgets
from traitlets import Unicode, validate
from IPython import display

logger=None

class ProgressImageWidget(widgets.DOMWidget):
    """
      ipywidget class to display incremental progress of training as an image
    """
    _view_name = Unicode('ProgressImageView').tag(sync=True)
    _view_module = Unicode('progress_image').tag(sync=True)
    value = Unicode().tag(sync=True)

In [2]:
%%javascript
require.undef('progress_image');

define('progress_image', ["jupyter-js-widgets"], function(widgets) {

    // Define the HelloView
    var ProgressImageView = widgets.DOMWidgetView.extend({
        // Render the view.
        render: function() {
            this.$img = $('<img />')
                .appendTo(this.$el);
        },
        
        update: function() {
            this.$img.attr('src', this.model.get('value'));
            return ProgressImageView.__super__.update.apply(this);
        },
        events: {"change": "handle_value_change"},
        
        handle_value_change: function(event) {
            this.model.set('value', this.$img.src);
            this.touch();
        },
        
    });

    return {
        ProgressImageView : ProgressImageView 
    }
});


<IPython.core.display.Javascript object>

### Plotter

Now for constructing the [datauri](https://en.wikipedia.org/wiki/Data_URI_scheme), which shows the progress graph, we employ another helper class called <code>Plotter</code> defined in <code>common/utils.ipynb</code>. We can construct a plotter with <code>xlabel, ylabel</code> and <code>title</code> parameters. An instance of a potter class, will return a png-datauri when the <code>plotter.plot()</code> is called.

We can also add many channels to the plot. For each channel we must supply an upperboumd on the number of samples that will be added to the channel. See <code>Plotter.add_channel</code>. All the channels will be shown on the same plot.

We should add as many samples to each channel as we please and if you call <code>plotter.plot()</code>, a pong datauri containing the plot will be returned.

In [3]:
import numpy as np
import io, base64
import os
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties



class Plotter(object):
    """
      A utility class to plot training/test data
      add_channel: Add as many channels as you want
      add_sample: Add as many samples to any channel
      plot: will return a dataurl containing a single plot 
    """
    
    format='PNG'
    def __init__(self,  **kwds):
        #need to have these keywords for initialization
        assert(kwds.get('xlabel'))
        assert(kwds.get('ylabel'))
        assert(kwds.get('title'))
        self.__dict__.update(kwds)
        #initialize empty extents
        self.extents=[np.inf, -np.inf, np.inf, -np.inf]
        self.channels={}
        pass
    
    #num_samples == upper bound on the number of samples that can be added for this channel    
    def add_channel(self, num_samples=-1, **kwds):
        assert(kwds.get('channel_name'))
        assert(kwds.get('legend'))
        channel = self.channels.setdefault(kwds['channel_name'], {})
        channel['plot_x'] = np.zeros(num_samples, dtype=np.float32)
        channel['plot_y'] = np.zeros_like( channel['plot_x']  )
        channel['legend'] = kwds['legend']
        channel['next_sample_index'] = 0
        
    #num_samples == upper bound on the number of samples that can be added for this channel    
    def change_channel(self,  **kwds):
        assert(kwds.get('channel_name_old'))
        assert(kwds.get('channel_name_new'))
        self.channels[kwds['channel_name_new']] = self.channels.pop(kwds['channel_name_old'])
    
    def del_channel(self,  **kwds):
        assert(kwds.get('channel_name'))
        self.channels.pop(kwds['channel_name'])        
    
    #add a sample to a channel
    def add_sample(self, x, y, channel_name=''):
        assert(channel_name)
        assert(self.channels.get(channel_name))
        channel = self.channels[channel_name]
        next_index = channel['next_sample_index']
        channel['plot_x'][next_index] = x
        channel['plot_y'][next_index] = y
        channel['next_sample_index'] += 1
        self.update_extents_(x, y)

        
    #internal routine to keep track of extents
    def update_extents_(self, x, y):
        self.extents[0 ] = 0 # min(x, self.extents[0])  
        self.extents[1 ] = max(x, self.extents[1])  
        self.extents[2 ] = 0 # min(y, self.extents[2])  
        self.extents[3 ] = 1 # max(y, self.extents[3])  

    def plot_core_(self):
        fontP = FontProperties()
        fontP.set_size('small')
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        #plot each channel
        for k,v in self.channels.iteritems():
            next_sample_index = v['next_sample_index']
            ax.plot(v['plot_x'][0:next_sample_index], v['plot_y'][0:next_sample_index], label=k)
        plt.legend( loc='lower left', prop=fontP)
        ax.set_title(self.title)
        ax.set_xlabel(self.ylabel)
        ax.set_xlabel(self.xlabel)
        #return the plot as a dataurl
        buf = io.BytesIO()    
        fig.savefig(buf, format=Plotter.format)
        buf.seek(0)
        fig.clear()
        plt.close(fig)
        return buf
    
    #plot routune
    def plot(self):
        buf = self.plot_core_()
        if  buf:
            dataurl = "data:image/" + Plotter.format + ";base64," + base64.b64encode(buf.read())
            return dataurl
        return None
    
    def plot_and_save_fig(self, savepath=''):
        assert(savepath)
        assert(os.path.splitext(savepath)[1].lower() == '.' + Plotter.format.lower())
        buf = self.plot_core_()
        with file(savepath, 'wb') as fp:
            fp.write(buf.read())
    
        

../### ImgGrid
In order to display the results of classification, we felt the need for dipslaying a grid of small images each titled with the appropriate class name (class name is result of the classification).. <code>ImgGrid</code> is another utility class intended for this purpose. Each instance of the <code>ImgGrid</code> can display up to <code>ImgGrid.limit</code>  images in a square grid with appropriate spaces for displaying the classification results. The default valu´for <code>ImgGrid.limit</code>  is 9, which results in a 3x3 grid. Each time we add an image to the grid, the combined image is regenerated and stored at <code>ImgGrid.cached_img_dataurl</code>.

Also, please note that each <code>ImgGrid</code> instance, only produces the grid image content as png-dataurl. For dispaying each <code>ImgGrid</code> instance, interactively as the content of the image list changes, requires  the use of an instance <code>ProgressImageWidget</code>. 

Usage

<code>
    grid_widget = ProgressImageWidget()
    grid = ImgGrid(limit=9)
    display(grid_widget)
    
    some loop
            .... lots of code
            got an image called ../data/foo.png classified as a class 'airplane'
            grid.add_img(img_path='../data/foo.png', data=dict(class_name='airplane'))
            if grid.cached_img_dataurl != None:
                grid_widget.value=grid.cached_img_dataurl
    
</code>
Here is an example result of the above code
![imggrid_demo](imggrid_demo.png)

In [5]:
from PIL import Image
import math
import matplotlib.font_manager as font_manager

class ImgGrid(object):
    def __init__(self, limit=9):
        self.img_dict={}
        #limit is the maximum jumber of images that can be displayed
        #by an insance of this class
        self.limit=limit
        #image generated
        self.cached_img_dataurl=None
        pass
    
    def is_full(self):
        #if the ImgGrid is full, please dont add any more images to it
        return len(self.img_dict.keys()) >= self.limit
    
    def add_img(self, img_path=None, data=None):
        if(len(self.img_dict.keys()) >= self.limit):
            raise IndexError('ImgGrid: cannlot add images, reached limit %d' % self.limit)
            pass
        self.img_dict[img_path] = data
        self.cached_img_dataurl = self.plot_()
      
    #core routine to make the image
    def plot_core_(self):
        #current number of images to display
        num_imgs = len(self.img_dict.keys())
        #arrange them in a square
        num_imgs_per_side=int(math.ceil(math.sqrt(num_imgs)))
        #if it is just one image then dont do anything
        if num_imgs_per_side <= 1:
            return None
        #get an appropriate size for your compbined grid image
        figsize = (8,8) if num_imgs_per_side == 3 else (5,5)
        #we are using a num_imgs_per_side x num_imgs_per_side grid
        fig, ax = plt.subplots(num_imgs_per_side, num_imgs_per_side, figsize=figsize)
        #we need some space between individual images in the grid
        #for displaying title
        fig.subplots_adjust(hspace=0, wspace=1)
        #get an iterator for the current set of images
        imgs_enum=enumerate(self.img_dict)        
        for i in range(num_imgs_per_side):
            for j in range(num_imgs_per_side):
                ax[i, j].xaxis.set_major_locator(plt.NullLocator())
                ax[i, j].yaxis.set_major_locator(plt.NullLocator())         
                im= None
                title=None
                try:
                    try :
                        _, rel_img_path=imgs_enum.next()
                        #img as an np-array
                        im= Image.open(rel_img_path)
                        im= np.asarray(im)
                        #get the classname 
                        title=self.img_dict[rel_img_path]['class_name']
                        #if there are multiple classes recognized
                        #put them in different ,lines
                        title='\n'.join(title.split(','))
                    except  IOError:
                        im=None
                        logger.error('cannot read image {p}'.format(p=rel_img_path))
                except StopIteration:
                    pass
                if im != None:                    
                    ax[i, j].imshow(im, cmap="bone")
                    ax[i, j].set_title(title, size=8 )
                    
        #get the image as a buffer
        buf = io.BytesIO()    
        fig.savefig(buf, format=Plotter.format)
        buf.seek(0)
        fig.clear()
        plt.close(fig)
        return buf

    #plot routine
    #if an image is generated, get it as a base-64 encided data url
    def plot_(self):
        buf = self.plot_core_()
        if buf:
            dataurl = "data:image/" + Plotter.format + ";base64," + base64.b64encode(buf.read())
            return dataurl
        return None
    
    def plot_and_save_fig(self, savepath=''):
        assert(savepath)
        assert(os.path.splitext(savepath)[1].lower() == '.' + Plotter.format.lower())
        buf = self.plot_core_()
        with file(savepath, 'wb') as fp:
            fp.write(buf.read())
            

###  ImgGridController

Since we may have more than the limit number of images to display, we employ a list of <code>ImgGrid</code> instances, along with their corresponding  <code>ProgressImageWidget</code> stacked vertically in a <code>widgets.VBox</code>.

One need to use only this class for displaying images in a grid if the number of images is expected to be greater than what a single <code>ImgGrid</code> can hold.

<code>
    grid_widget = ProgressImageWidget()
    grid = ImgGrid(limit=9)
    display(grid_widget)

    some loop
        .... lots of code
        got an image called ../data/foo.png classified as a class 'airplane'
        grid.add_img(img_path='../data/foo.png', data=dict(class_name='airplane'))
        if grid.cached_img_dataurl != None:
            grid_widget.value=grid.cached_img_dataurl
</code>
Here is an example output
![imggridcontroller_demo](imggridcontroller_demo.png)

In [ ]:

            
class ImgGridController(object):
    def __init__(self):
        #list of ImgGrid-s
        self.imggrid_list=[]
        #coresponding list of ProgressImageWidgets
        self.top_level=widgets.VBox(children=tuple())
    
    def add_img(self, img_path=None, data=None):
        #ensure that for each ImgGrid, there is a ProgressImageWidget
        assert(len(self.imggrid_list) == len(self.top_level.children))
        is_added=False
        #look for the next free IngGrid to add this image
        for l in self.imggrid_list:
            if not l.is_full():
                l.add_img(img_path=img_path, data=data)
                is_added=True
                break
        #If all ImgGrids are full, then add a new ImgGrid,
        #which is displayed at the end of the stack
        #make sure you add a corresponding ProgressImageWidgets well.
        if not is_added:
            new_grid=ImgGrid(limit=9)
            new_grid.add_img(img_path=img_path, data=data)
            ch_list=list(self.top_level.children)
            ch_list.append(ProgressImageWidget())
            self.top_level.children = tuple(ch_list)
            self.imggrid_list.append(new_grid)
        #progressimagewidgets are assigned with the current content of 
        #corresponding ImgGrid-s
        for w, g in zip(self.top_level.children, self.imggrid_list):
            if g.cached_img_dataurl != None:
                w.value = g.cached_img_dataurl
    
            
    